In [183]:
# Importing Libraries:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler

# Data Cleaning & Pre-processing:

In [5]:
# 1. Loading dataset:
uber_data= pd.read_csv(r"C:\Users\hp\Desktop\Dataset_Uber Traffic.csv")
uber_data.head()

,DateTime,Junction,Vehicles,ID
0,01-11-2015 0.00,1,15,20151101001
1,01-11-2015 1.00,1,13,20151101011
2,01-11-2015 2.00,1,10,20151101021
3,01-11-2015 3.00,1,7,20151101031
4,01-11-2015 4.00,1,9,20151101041


In [6]:
# Initial Inspections:
uber_data.shape

(48120, 4)

In [7]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DateTime  48120 non-null  object
 1   Junction  48120 non-null  int64 
 2   Vehicles  48120 non-null  int64 
 3   ID        48120 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [8]:
uber_data.describe()

,Junction,Vehicles,ID
count,48120.000000,48120.000000,4.812000e+04
mean,2.180549,22.791334,2.016330e+10
std,0.966955,20.750063,5.944854e+06
min,1.000000,1.000000,2.015110e+10
25%,1.000000,9.000000,2.016042e+10
50%,2.000000,15.000000,2.016093e+10
75%,3.000000,29.000000,2.017023e+10
max,4.000000,180.000000,2.017063e+10


In [9]:
# 2. Clean Data:
# Checking null values
uber_data.isnull().sum()

DateTime    0
Junction    0
Vehicles    0
ID          0
dtype: int64

In [10]:
# Dropping duplicate values:
uber_data.drop_duplicates(inplace=True)

In [11]:
uber_data.shape

(48120, 4)

In [12]:
# Correct Data types:
uber_data['DateTime'] = pd.to_datetime(uber_data['DateTime'],format='%d-%m-%Y %H.%M')
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48120 entries, 0 to 48119
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DateTime  48120 non-null  datetime64[ns]
 1   Junction  48120 non-null  int64         
 2   Vehicles  48120 non-null  int64         
 3   ID        48120 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 1.5 MB


In [13]:
# 3. Aggregate Traffic Data (Hourly per Junction):
# Group by hourly DateTime and Junction and aggregate vehicle count
traffic_hourly = uber_data.groupby([pd.Grouper(key='DateTime', freq='H'), 'Junction'])['Vehicles'].sum().reset_index()

In [14]:
traffic_hourly.head()

,DateTime,Junction,Vehicles
0,2015-11-01 00:00:00,1,15
1,2015-11-01 00:00:00,2,6
2,2015-11-01 00:00:00,3,9
3,2015-11-01 01:00:00,1,13
4,2015-11-01 01:00:00,2,6


In [15]:
# Pre-process the data:
# Normalize or standardize the data to facilitate comparison across different time periods and junctions.
scaler = StandardScaler()
traffic_hourly['Vehicles_scaled'] = scaler.fit_transform(traffic_hourly[['Vehicles']])
traffic_hourly.head()

,DateTime,Junction,Vehicles,Vehicles_scaled
0,2015-11-01 00:00:00,1,15,-0.375489
1,2015-11-01 00:00:00,2,6,-0.809227
2,2015-11-01 00:00:00,3,9,-0.664648
3,2015-11-01 01:00:00,1,13,-0.471875
4,2015-11-01 01:00:00,2,6,-0.809227


# Feature Engineering and Selection:

In [17]:
# Time-based features
traffic_hourly['Hour'] = traffic_hourly['DateTime'].dt.hour
traffic_hourly['DayOfWeek'] = traffic_hourly['DateTime'].dt.dayofweek
traffic_hourly['Month'] = traffic_hourly['DateTime'].dt.month
traffic_hourly['IsWeekend'] = traffic_hourly['DayOfWeek'].isin([5, 6]).astype(int)

# Sort values to create lag features
traffic_hourly = traffic_hourly.sort_values(['Junction', 'DateTime'])

# Lag features
traffic_hourly['Lag_1'] = traffic_hourly.groupby('Junction')['Vehicles'].shift(1)
traffic_hourly['Lag_2'] = traffic_hourly.groupby('Junction')['Vehicles'].shift(2)
traffic_hourly['Lag_24'] = traffic_hourly.groupby('Junction')['Vehicles'].shift(24)

traffic_hourly.head()

,DateTime,Junction,Vehicles,Vehicles_scaled,Hour,DayOfWeek,Month,IsWeekend,Lag_1,Lag_2,Lag_24
0,2015-11-01 00:00:00,1,15,-0.375489,0,6,11,1,NaN,NaN,NaN
3,2015-11-01 01:00:00,1,13,-0.471875,1,6,11,1,15.0,NaN,NaN
6,2015-11-01 02:00:00,1,10,-0.616454,2,6,11,1,13.0,15.0,NaN
9,2015-11-01 03:00:00,1,7,-0.761034,3,6,11,1,10.0,13.0,NaN
12,2015-11-01 04:00:00,1,9,-0.664648,4,6,11,1,7.0,10.0,NaN


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Drop NA due to lag creation
df_model = traffic_hourly.dropna()

# Feature columns
feature_cols = ['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'Lag_1', 'Lag_2', 'Lag_24']
X = df_model[feature_cols]
y = df_model['Vehicles']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit RandomForest to get feature importance
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Create feature importance DataFrame
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending=False)


feature_importance.head(8)

,Feature,Importance
4,Lag_1,0.942443
6,Lag_24,0.017474
0,Hour,0.016546
5,Lag_2,0.011259
2,Month,0.006071
1,DayOfWeek,0.005165
3,IsWeekend,0.001043


### Weather Dataset:

In [308]:
# Weather Dataset:
weather_data= pd.read_csv(r"C:\Users\hp\Desktop\rainfall.csv")
weather_data.head()

,datetime,temp,dew,humidity,sealevelpressure,winddir,solarradiation,windspeed,precipprob,preciptype
0,01-01-2016,28.4,11.9,37.8,1016.4,147.5,216.1,16.6,0,0
1,02-01-2016,26.8,13.0,44.8,1017.2,110.3,215.7,16.6,0,0
2,03-01-2016,25.5,14.6,52.8,1015.7,145.3,221.1,18.4,0,0
3,04-01-2016,26.4,13.1,46.6,1015.3,126.9,216.2,16.6,0,0
4,05-01-2016,27.1,13.5,44.9,1014.4,125.5,208.1,16.6,0,0


In [310]:
weather_data.shape

(1781, 10)

In [312]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   datetime          1781 non-null   object 
 1   temp              1781 non-null   float64
 2   dew               1781 non-null   float64
 3   humidity          1781 non-null   float64
 4   sealevelpressure  1781 non-null   float64
 5   winddir           1781 non-null   float64
 6   solarradiation    1781 non-null   float64
 7   windspeed         1781 non-null   float64
 8   precipprob        1781 non-null   int64  
 9   preciptype        1781 non-null   int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 139.3+ KB


In [314]:
weather_data = weather_data.drop(['solarradiation','winddir'], axis=1)

In [316]:
weather_data.isnull().sum()

datetime            0
temp                0
dew                 0
humidity            0
sealevelpressure    0
windspeed           0
precipprob          0
preciptype          0
dtype: int64

In [318]:
weather_data.drop_duplicates(inplace=True)

In [320]:
weather_data.shape

(1781, 8)

In [322]:
weather_data = weather_data.rename(columns={'datetime': 'DateTime'})

In [324]:
weather_data['DateTime'] = pd.to_datetime(weather_data['DateTime'],format='%d-%m-%Y')
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DateTime          1781 non-null   datetime64[ns]
 1   temp              1781 non-null   float64       
 2   dew               1781 non-null   float64       
 3   humidity          1781 non-null   float64       
 4   sealevelpressure  1781 non-null   float64       
 5   windspeed         1781 non-null   float64       
 6   precipprob        1781 non-null   int64         
 7   preciptype        1781 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 111.4 KB


### Event Dataset:

In [327]:
# Event Dataset:
Event_data= pd.read_csv(r"C:\Users\hp\Desktop\Event_dataset.csv")
Event_data.head()

,EventID,EventType,EventName,Date,Location,Description,ExpectedAttendance,Source
0,1,Sports,Standard Chartered Mumbai Marathon,18-01-2015,Mumbai,Annual city marathon event,39000.0,https://mumbaimarathon.com
1,2,Public Holiday,Republic Day,26-01-2015,All Mumbai,National holiday,NaN,https://india.gov.in
2,3,Concert,Sunburn Arena Mumbai,14-02-2015,Mahalaxmi Racecourse,EDM concert featuring international DJs,12000.0,https://sunburn.in
3,4,Demonstration,Women's Rights March,08-03-2015,Marine Drive,Public demonstration for International Women's...,3000.0,https://timesofindia.indiatimes.com
4,5,Public Holiday,Holi,06-03-2015,All Mumbai,Hindu festival of colors,NaN,https://india.gov.in


In [329]:
Event_data.shape

(50, 8)

In [331]:
Event_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EventID             50 non-null     int64  
 1   EventType           50 non-null     object 
 2   EventName           50 non-null     object 
 3   Date                50 non-null     object 
 4   Location            50 non-null     object 
 5   Description         50 non-null     object 
 6   ExpectedAttendance  33 non-null     float64
 7   Source              50 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.3+ KB


In [333]:
Event_data['Date'] = pd.to_datetime(Event_data['Date'],format='%d-%m-%Y')
Event_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   EventID             50 non-null     int64         
 1   EventType           50 non-null     object        
 2   EventName           50 non-null     object        
 3   Date                50 non-null     datetime64[ns]
 4   Location            50 non-null     object        
 5   Description         50 non-null     object        
 6   ExpectedAttendance  33 non-null     float64       
 7   Source              50 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 3.3+ KB


In [335]:
# Rename column name:
Event_data = Event_data.rename(columns={'Date': 'DateTime'})

In [337]:
Event_data.isnull().sum()

EventID                0
EventType              0
EventName              0
DateTime               0
Location               0
Description            0
ExpectedAttendance    17
Source                 0
dtype: int64

In [339]:
Event_data.describe()

,EventID,DateTime,ExpectedAttendance
count,50.00000,50,33.000000
mean,25.50000,2016-04-21 22:33:36,15560.606061
min,1.00000,2015-01-18 00:00:00,2000.000000
25%,13.25000,2015-09-17 18:00:00,5000.000000
50%,25.50000,2016-04-25 12:00:00,9000.000000
75%,37.75000,2016-12-01 18:00:00,17000.000000
max,50.00000,2017-08-20 00:00:00,70000.000000
std,14.57738,NaN,15808.580332


In [341]:
# Missing values:
median_value = Event_data['ExpectedAttendance'].median()
Event_data['ExpectedAttendance'].fillna(median_value, inplace=True)
Event_data.head()

,EventID,EventType,EventName,DateTime,Location,Description,ExpectedAttendance,Source
0,1,Sports,Standard Chartered Mumbai Marathon,2015-01-18,Mumbai,Annual city marathon event,39000.0,https://mumbaimarathon.com
1,2,Public Holiday,Republic Day,2015-01-26,All Mumbai,National holiday,9000.0,https://india.gov.in
2,3,Concert,Sunburn Arena Mumbai,2015-02-14,Mahalaxmi Racecourse,EDM concert featuring international DJs,12000.0,https://sunburn.in
3,4,Demonstration,Women's Rights March,2015-03-08,Marine Drive,Public demonstration for International Women's...,3000.0,https://timesofindia.indiatimes.com
4,5,Public Holiday,Holi,2015-03-06,All Mumbai,Hindu festival of colors,9000.0,https://india.gov.in


In [343]:
# Dropping 'Source' column:
Event_data.drop('Source', axis=1, inplace=True)
Event_data.head()

,EventID,EventType,EventName,DateTime,Location,Description,ExpectedAttendance
0,1,Sports,Standard Chartered Mumbai Marathon,2015-01-18,Mumbai,Annual city marathon event,39000.0
1,2,Public Holiday,Republic Day,2015-01-26,All Mumbai,National holiday,9000.0
2,3,Concert,Sunburn Arena Mumbai,2015-02-14,Mahalaxmi Racecourse,EDM concert featuring international DJs,12000.0
3,4,Demonstration,Women's Rights March,2015-03-08,Marine Drive,Public demonstration for International Women's...,3000.0
4,5,Public Holiday,Holi,2015-03-06,All Mumbai,Hindu festival of colors,9000.0


In [345]:
Event_data.drop_duplicates(inplace=True)
Event_data.shape

(50, 7)

## Merging all Datasets:

In [348]:
# Merge traffic and weather data on timestamp
merged_df = pd.merge(uber_data, weather_data, on="DateTime", how="left")

In [350]:
# Convert events to hourly format using one-hot encoding
Event_data['event_flag'] = 1
event_features = pd.get_dummies(Event_data['EventType'])

In [386]:
# Join with timestamp
events_encoded = pd.concat([Event_data['DateTime'], event_features], axis=1)

# Resample to hourly granularity, summing up events
events_hourly = events_encoded.groupby('DateTime').sum().reset_index()

# Merge all together
final_data = pd.merge(merged_df, events_hourly, on="DateTime", how="left")

# Fill NaNs in event columns with 0 (no event)
event_cols = event_features.columns
final_data[event_cols] = final_data[event_cols].fillna(0)
final_data.head()

,DateTime,Junction,Vehicles,ID,temp,dew,humidity,sealevelpressure,windspeed,precipprob,preciptype,Concert,Demonstration,Public Holiday,Sports
0,2015-11-01 00:00:00,1,15,20151101001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
1,2015-11-01 01:00:00,1,13,20151101011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
2,2015-11-01 02:00:00,1,10,20151101021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,2015-11-01 03:00:00,1,7,20151101031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,2015-11-01 04:00:00,1,9,20151101041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [388]:
# Remove duplicates:
final_data.drop_duplicates(inplace=True)

In [390]:
final_data.isnull().sum()

DateTime                0
Junction                0
Vehicles                0
ID                      0
temp                46298
dew                 46298
humidity            46298
sealevelpressure    46298
windspeed           46298
precipprob          46298
preciptype          46298
Concert                 0
Demonstration           0
Public Holiday          0
Sports                  0
dtype: int64

In [392]:
# Fill remaining with mean (for weather columns)
weather_cols = ['temp', 'dew', 'precipprob', 'sealevelpressure', 'humidity', 'windspeed', 'preciptype']
for col in weather_cols:
    final_data[col] = final_data[col].fillna(final_data[col].mean())

In [394]:
final_data.isnull().sum()

DateTime            0
Junction            0
Vehicles            0
ID                  0
temp                0
dew                 0
humidity            0
sealevelpressure    0
windspeed           0
precipprob          0
preciptype          0
Concert             0
Demonstration       0
Public Holiday      0
Sports              0
dtype: int64

In [402]:
scaler = StandardScaler()

# Define columns to scale (exclude timestamp and traffic volume initially)
features_to_scale = ['temp', 'dew', 'precipprob', 'sealevelpressure', 'humidity', 'windspeed', 'preciptype']
final_data[features_to_scale] = scaler.fit_transform(final_data[features_to_scale])
final_data[features_to_scale].head()

,temp,dew,precipprob,sealevelpressure,humidity,windspeed,preciptype
0,-3.606033e-16,-1.092940e-16,-1.982599e-17,-2.041260e-15,-4.018288e-17,1.243110e-16,-5.715541e-18
1,-3.606033e-16,-1.092940e-16,-1.982599e-17,-2.041260e-15,-4.018288e-17,1.243110e-16,-5.715541e-18
2,-3.606033e-16,-1.092940e-16,-1.982599e-17,-2.041260e-15,-4.018288e-17,1.243110e-16,-5.715541e-18
3,-3.606033e-16,-1.092940e-16,-1.982599e-17,-2.041260e-15,-4.018288e-17,1.243110e-16,-5.715541e-18
4,-3.606033e-16,-1.092940e-16,-1.982599e-17,-2.041260e-15,-4.018288e-17,1.243110e-16,-5.715541e-18


## Final Dataset after merging Traffic, Weather & Event dataset:

In [418]:
final_data.to_csv("final_traffic_dataset.csv", index=False)
final_data.head()

,DateTime,Junction,Vehicles,ID,temp,dew,humidity,sealevelpressure,windspeed,precipprob,preciptype,Concert,Demonstration,Public Holiday,Sports
0,2015-11-01 00:00:00,1,15,20151101001,-3.606033e-16,-1.092940e-16,-4.018288e-17,-2.041260e-15,1.243110e-16,-1.982599e-17,-5.715541e-18,1.0,0.0,0.0,0.0
1,2015-11-01 01:00:00,1,13,20151101011,-3.606033e-16,-1.092940e-16,-4.018288e-17,-2.041260e-15,1.243110e-16,-1.982599e-17,-5.715541e-18,0.0,0.0,0.0,0.0
2,2015-11-01 02:00:00,1,10,20151101021,-3.606033e-16,-1.092940e-16,-4.018288e-17,-2.041260e-15,1.243110e-16,-1.982599e-17,-5.715541e-18,0.0,0.0,0.0,0.0
3,2015-11-01 03:00:00,1,7,20151101031,-3.606033e-16,-1.092940e-16,-4.018288e-17,-2.041260e-15,1.243110e-16,-1.982599e-17,-5.715541e-18,0.0,0.0,0.0,0.0
4,2015-11-01 04:00:00,1,9,20151101041,-3.606033e-16,-1.092940e-16,-4.018288e-17,-2.041260e-15,1.243110e-16,-1.982599e-17,-5.715541e-18,0.0,0.0,0.0,0.0
